<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第三章：编码注意力机制

本notebook会用到以下第三方库

In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.2.1+cu118


- 本章节覆盖LLMs的引擎--注意力机制

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">
</p>

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">
</p>

## 3.1 建模长序列的问题
- 本小结没有代码
- 在不同语种文本翻译任务中，因为不同语言的语法结构不同导致逐词翻译是不可行的

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">
</p>

- 在transformer模型出现之前，编码器-解码器RNN通常用于机器翻译任务
- 在这种设置中，编码器处理源语言的标记序列，使用隐藏状态（神经网络内部的一种中间层）来生成整个输入序列的浓缩表示

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="600px">
</p>

## 3.2 使用注意力机制捕捉数据之间的依赖
- 本小结没有代码
- 通过注意力机制，网络的文本生成解码器部分能够选择性地访问所有输入标记，这意味着在生成特定输出标记时，某些输入标记比其他标记更重要

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="600px">
</p>

- 自注意力在transformer中是一种旨在增强输入表示的技术，它使序列中的每个位置都能与同一序列中的所有其他位置进行交互并确定它们的相关性

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">
</p>

## 3.3 通过自注意力关注输入的不同部分

### 3.3.1 一个不带可训练权重的简单自注意力机制
- 本节解释了一个非常简化的自注意力变体，它不包含任何可训练的权重
- 这纯粹是为了说明目的，而不是transformer中使用的注意力机制
- 下一节，3.3.2节，将扩展这个简单的注意力机制来实现真正的自注意力机制
- 假设我们有一个输入序列$x^{(1)}$到$x^{(T)}$
  - 输入是一段文本（例如，一个句子如"Your journey starts with one step"），已经按照第2章所述转换为标记嵌入
  - 例如，$x^{(1)}$是表示单词"Your"的d维向量，以此类推
- **目标：**为输入序列$x^{(1)}$到$x^{(T)}$中的每个输入序列元素$x^{(i)}$计算上下文向量$z^{(i)}$（其中$z$和$x$具有相同的维度）
    - 上下文向量$z^{(i)}$是输入$x^{(1)}$到$x^{(T)}$的加权和
    - 上下文向量对特定输入是"上下文"特定的
      - 不用$x^{(i)}$作为任意输入标记的占位符，关于第二个输入$x^{(2)}$
      - 继续使用具体示例，不用占位符$z^{(i)}$，考虑第二个输出上下文向量$z^{(2)}$
      - 第二个上下文向量$z^{(2)}$是所有输入$x^{(1)}$到$x^{(T)}$的加权和，权重是相对于第二个输入元素$x^{(2)}$的
      - 注意力权重是决定在计算$z^{(2)}$时每个输入元素贡献多少的权重
      - 简而言之，可以将$z^{(2)}$视为$x^{(2)}$的修改版本，它还包含了与给定任务相关的所有其他输入元素的信息

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">
</p>

- （请注意，此图中的数字截断为小数点后一位，以减少视觉混乱；类似地，其他图表也可能包含截断的值）

- 按照惯例，未归一化的注意力权重被称为"注意力分数"，而归一化后和为1的注意力分数则被称为"注意力权重"

- 下面的代码逐步解释了上图中的过程

<br>

- 第一步：计算未归一化的注意力分数 $\omega$
- 假设使用第二个input token作为query，即 $q^{(2)} = x^{(2)}$，通过以下点乘操作计算为归一化分数
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上面的 $\omega$ 是希腊字母"omega"，用来表示未归一化的注意力分数
    - 下标"21"在$\omega_{21}$中表示输入序列元素2被用作针对输入序列元素1的查询

- 假设有以下输入句子，已经按照第3章所述嵌入到3维向量中（这里为了说明目的使用非常小的嵌入维度，这样它可以在页面上不换行地显示）

In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- （在本书中遵循常见的机器学习和深度学习惯例，其中训练样本表示为行，特征值表示为列；在上面显示的张量中，每一行代表一个词，每一列代表一个嵌入维度）

- 本节的主要目标是演示如何使用第二个输入序列$x^{(2)}$作为查询来计算上下文向量$z^{(2)}$

- 图中描述了这个过程的初始步骤，即通过点积操作计算$x^{(2)}$与所有其他输入元素之间的注意力分数ω

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">
</p>

- 使用输入序列元素2，$x^{(2)}$，作为示例来计算上下文向量$z^{(2)}$；在本节后面，将推广这一点来计算所有上下文向量。

- 第一步是通过计算查询$x^{(2)}$与所有其他输入标记之间的点积来计算未归一化的注意力分数

In [4]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 附注：点积本质上是将两个向量按元素相乘并对结果求和的简写

In [5]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **步骤2**：归一化未归一化的注意力分数（"omega"，$\omega$），使它们的总和为1
- 这里是一种将未归一化注意力分数归一化为总和为1的简单方法（这是一种惯例，有助于解释，并且对训练稳定性很重要）

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">
</p>

In [6]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实践中，使用softmax函数进行归一化是常见且推荐的，它更善于处理极端值，并且在训练期间具有更理想的梯度特性
- 这里是一个用于缩放的softmax函数的简单实现，它也将向量元素归一化，使它们的总和为1

In [7]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上面的简单实现可能会因为大或小的输入值而遇到数值不稳定问题，这是由于溢出和下溢问题
- 因此，在实践中，建议使用PyTorch的softmax实现，它已经为性能进行了高度优化

In [8]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤3**：计算上下文向量$z^{(2)}$，方法是将嵌入的输入标记$x^{(i)}$与注意力权重相乘，然后对结果向量求和

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">
</p>

In [9]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 对所有输入tokens计算注意力权重

#### 概括所有输入序列tokens

- 上面已计算了输入2的注意力权重和上下文向量（如下图中突出显示的行所示）
- 接下来，将这种计算推广到计算所有注意力权重和上下文向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">
</p>

- （请注意，此图中的数字截断为小数点后两位，以减少视觉混乱；每行中的值应加起来等于1.0或100%；类似地，其他图中的数字也被截断）

- 自注意力过程从计算注意力分数开始，然后将其归一化以得到总和为1的注意力权重
- 这些注意力权重随后通过输入的加权求和来生成上下文向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">
</p>

- 将之前的**步骤1**应用于所有成对元素，计算未归一化的注意力分数矩阵：

In [10]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 可以通过矩阵乘法更高效地实现与上述相同的结果

In [11]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的**步骤2**类似，对每一行进行归一化，使每行中的值总和为1

In [12]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每行中的值确实总和为1

In [13]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用之前的**步骤3**来计算所有上下文向量

In [14]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为合理性检查，之前计算的上下文向量$z^{(2)} = [0.4419, 0.6515, 0.5683]$可以在上面的第2行找到

In [15]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])
